In [32]:
import pandas as pd

data = pd.read_csv("/home/ictc/Desktop/078msdsa016/covid_sentiment/CovidNepaliTweet_Research/sentiment_youtube_comment/collected_final_data.csv")

In [33]:
import torch
import numpy as np 
import pandas as pd 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from scipy.spatial.distance import cosine 
import tokenizers 
import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# import snowballstemmer
import numpy
import os 
import re
import json
import pickle 

model = AutoModelForMaskedLM.from_pretrained("Sakonii/deberta-base-nepali", output_hidden_states = True, return_dict = True, output_attentions = True)

tokenizers = AutoTokenizer.from_pretrained("Sakonii/deberta-base-nepali")

In [34]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('Sakonii/deberta-base-nepali')

def get_bert_embedding_sentence(sentence):

    query_embedding = model.encode(sentence)

    return query_embedding

No sentence-transformers model found with name /home/ictc/.cache/torch/sentence_transformers/Sakonii_deberta-base-nepali. Creating a new one with MEAN pooling.


In [35]:
data['word_embeddings'] = data['text'].apply(get_bert_embedding_sentence)

In [36]:
data.to_csv("embeddings_data_nepali_BERT.csv")

In [37]:
X,y = data['word_embeddings'], data['label']

In [38]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state = 420)

In [39]:
svc = SVC(probability=True)

In [40]:
svc.fit(train_X.tolist(), train_y)
#svc.fit(train_X, train_y)

SVC(probability=True)

In [41]:
svc_pred = svc.predict(test_X.tolist())
# svc_pred = svc.predict(test_X)

In [42]:
print(confusion_matrix(test_y, svc_pred))

[[433  82]
 [ 79 618]]


In [43]:
print(classification_report(test_y, svc_pred))

              precision    recall  f1-score   support

           0       0.85      0.84      0.84       515
           1       0.88      0.89      0.88       697

    accuracy                           0.87      1212
   macro avg       0.86      0.86      0.86      1212
weighted avg       0.87      0.87      0.87      1212



In [44]:
accuracy_score(test_y, svc_pred)

0.8671617161716172

In [45]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth=2, random_state=0)
random_forest.fit(train_X.tolist(), train_y)



RandomForestClassifier(max_depth=2, random_state=0)

In [46]:
random_pred = random_forest.predict(test_X.tolist())
print(accuracy_score(test_y, random_pred))

print(confusion_matrix(test_y, random_pred))

print(classification_report(test_y, random_pred))

0.7615511551155115
[[267 248]
 [ 41 656]]
              precision    recall  f1-score   support

           0       0.87      0.52      0.65       515
           1       0.73      0.94      0.82       697

    accuracy                           0.76      1212
   macro avg       0.80      0.73      0.73      1212
weighted avg       0.79      0.76      0.75      1212



# Naive Bayes

In [47]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_X.tolist(), train_y)


GaussianNB()

In [48]:
gnb_pred = gnb.predict(test_X.tolist())
print(accuracy_score(test_y, gnb_pred))

print(confusion_matrix(test_y, gnb_pred))

print(classification_report(test_y, gnb_pred))

0.8044554455445545
[[440  75]
 [162 535]]
              precision    recall  f1-score   support

           0       0.73      0.85      0.79       515
           1       0.88      0.77      0.82       697

    accuracy                           0.80      1212
   macro avg       0.80      0.81      0.80      1212
weighted avg       0.81      0.80      0.81      1212



# Ada Boost

In [49]:
from sklearn.ensemble import AdaBoostClassifier
ada_boost_clf = AdaBoostClassifier(n_estimators=100, random_state=0)
ada_boost_clf.fit(train_X.tolist(), train_y)


AdaBoostClassifier(n_estimators=100, random_state=0)

In [50]:
adaboost_pred = ada_boost_clf.predict(test_X.tolist())
print(accuracy_score(test_y, adaboost_pred))

print(confusion_matrix(test_y, adaboost_pred))

print(classification_report(test_y, adaboost_pred))

0.8308580858085809
[[401 114]
 [ 91 606]]
              precision    recall  f1-score   support

           0       0.82      0.78      0.80       515
           1       0.84      0.87      0.86       697

    accuracy                           0.83      1212
   macro avg       0.83      0.82      0.83      1212
weighted avg       0.83      0.83      0.83      1212



# Decision Tree

In [51]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree.fit(train_X.tolist(), train_y)


DecisionTreeClassifier(max_depth=2, random_state=0)

In [52]:
decision_pred = decision_tree.predict(test_X.tolist())
print(accuracy_score(test_y, decision_pred))

print(confusion_matrix(test_y, decision_pred))

print(classification_report(test_y, decision_pred))

0.7384488448844885
[[312 203]
 [114 583]]
              precision    recall  f1-score   support

           0       0.73      0.61      0.66       515
           1       0.74      0.84      0.79       697

    accuracy                           0.74      1212
   macro avg       0.74      0.72      0.72      1212
weighted avg       0.74      0.74      0.73      1212



# Extra Trees Classifier

In [53]:
from sklearn.ensemble import ExtraTreesClassifier
etc_clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
etc_clf.fit(train_X.tolist(), train_y)


ExtraTreesClassifier(random_state=0)

In [54]:
etc_pred = etc_clf.predict(test_X.tolist())
print(accuracy_score(test_y, etc_pred))

print(confusion_matrix(test_y, etc_pred))

print(classification_report(test_y, etc_pred))

0.8663366336633663
[[410 105]
 [ 57 640]]
              precision    recall  f1-score   support

           0       0.88      0.80      0.84       515
           1       0.86      0.92      0.89       697

    accuracy                           0.87      1212
   macro avg       0.87      0.86      0.86      1212
weighted avg       0.87      0.87      0.87      1212



# Logistic Regression

In [55]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(train_X.tolist(), train_y)

/home/ictc/anaconda3/envs/shushanta_nepali_gpt/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=100000.0)

In [56]:
lr_pred = logreg.predict(test_X.tolist())
print(accuracy_score(test_y, lr_pred))

print(confusion_matrix(test_y, lr_pred))

print(classification_report(test_y, lr_pred))

0.8506600660066007
[[427  88]
 [ 93 604]]
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       515
           1       0.87      0.87      0.87       697

    accuracy                           0.85      1212
   macro avg       0.85      0.85      0.85      1212
weighted avg       0.85      0.85      0.85      1212



# KNN

In [57]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X.tolist(), train_y)

KNeighborsClassifier(n_neighbors=3)

In [58]:
neigh_pred = neigh.predict(test_X.tolist())
print(accuracy_score(test_y, neigh_pred))

print(confusion_matrix(test_y, neigh_pred))

print(classification_report(test_y, neigh_pred))

0.8283828382838284
[[401 114]
 [ 94 603]]
              precision    recall  f1-score   support

           0       0.81      0.78      0.79       515
           1       0.84      0.87      0.85       697

    accuracy                           0.83      1212
   macro avg       0.83      0.82      0.82      1212
weighted avg       0.83      0.83      0.83      1212



# MLP

In [59]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=300)
mlp.fit(train_X.tolist(), train_y)

MLPClassifier(max_iter=300, random_state=1)

In [60]:
mlp_pred = mlp.predict(test_X.tolist())
print(accuracy_score(test_y, mlp_pred))

print(confusion_matrix(test_y, mlp_pred))

print(classification_report(test_y, mlp_pred))

0.8762376237623762
[[445  70]
 [ 80 617]]
              precision    recall  f1-score   support

           0       0.85      0.86      0.86       515
           1       0.90      0.89      0.89       697

    accuracy                           0.88      1212
   macro avg       0.87      0.87      0.87      1212
weighted avg       0.88      0.88      0.88      1212

